In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


In [23]:
# Define constants
IMAGE_SIZE = (128, 128)  # Resize all images to this size
CLASSES = ["daisy", "dandelion", "rose", "sunflower", "tulip"]
NUM_CLASSES = len(CLASSES)
DATASET_PATH = "flowers/"  # Replace with your dataset path
BATCH_SIZE = 32
EPOCHS = 20

In [26]:
# Step 1: Load and preprocess data

from glob import glob
import os
import numpy as np
from PIL import Image
def load_data(dataset_path, image_size, classes):
    images = []
    labels = []
    for label, flower_class in enumerate(classes):
        folder = os.path.join(dataset_path, flower_class)
        #print(folder)
        for file_path in glob(os.path.join(folder, "*.jpg")):
            try:
                img = Image.open(file_path).convert("RGB")  # Load as RGB
                img = img.resize(image_size)  # Resize
                images.append(np.array(img))
                labels.append(label)
                #print(labels)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
    return np.array(images), np.array(labels)


In [27]:
X, y = load_data(DATASET_PATH, IMAGE_SIZE, CLASSES)

In [28]:
print(y)

[0 0 0 ... 4 4 4]


In [29]:
## Onehot encode 'Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
df = pd.DataFrame()
df['Tag'] = pd.DataFrame(y)

geo_encoder=onehot_encoder_geo.fit_transform(df[['Tag']]).toarray()
print(geo_encoder)


[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [31]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, geo_encoder, test_size=0.2, random_state=42)



In [32]:
# Step 2: Define the CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


BASE_MODEL_WEIGHTS = 'imagenet'  # Use pre-trained weights

base_model = ResNet50(weights=BASE_MODEL_WEIGHTS, include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

for layer in base_model.layers:
    layer.trainable = False
x = base_model.output  # Extract features from ResNet-50
x = GlobalAveragePooling2D()(x)  # Pooling to reduce dimensionality
x = Dense(128, activation='relu')(x)  # Optional dense layer
x = Dense(NUM_CLASSES, activation='softmax')(x)  # Final classification layer
# Create the complete model
model = Model(inputs=base_model.input, outputs=x)


In [10]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 134, 134, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 64, 64, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 64, 64, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [33]:
# Step 4: Compile the model
from tensorflow.keras.optimizers import SGD

model.compile(optimizer=SGD(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
# Step 5: Train the model
# Assume X_train, y_train, X_test, y_test are preprocessed and ready
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)

Epoch 1/10
108/108 [==============================] - 52s 484ms/step - loss: 0.5576 - accuracy: 0.7900 - val_loss: 0.5274 - val_accuracy: 0.8079
Epoch 2/10
108/108 [==============================] - 55s 514ms/step - loss: 0.4094 - accuracy: 0.8485 - val_loss: 0.4622 - val_accuracy: 0.8345
Epoch 3/10
108/108 [==============================] - 59s 543ms/step - loss: 0.3326 - accuracy: 0.8839 - val_loss: 0.4358 - val_accuracy: 0.8356
Epoch 4/10
108/108 [==============================] - 58s 536ms/step - loss: 0.2766 - accuracy: 0.9015 - val_loss: 0.4446 - val_accuracy: 0.8380
Epoch 5/10
108/108 [==============================] - 63s 586ms/step - loss: 0.2354 - accuracy: 0.9247 - val_loss: 0.4082 - val_accuracy: 0.8484
Epoch 6/10
108/108 [==============================] - 60s 557ms/step - loss: 0.1998 - accuracy: 0.9409 - val_loss: 0.3957 - val_accuracy: 0.8530
Epoch 7/10
108/108 [==============================] - 56s 520ms/step - loss: 0.1739 - accuracy: 0.9542 - val_loss: 0.4025 - val_ac

In [36]:
# Step 6: Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Step 7: Save the model
model.save("resnet50_flower_classifier.h5")

27/27 [==============================] - 8s 306ms/step - loss: 0.4006 - accuracy: 0.8542
Test Accuracy: 85.42%


c:\Users\noams\Desktop\MyProjects\FlowerRecognition\flowerRecognitionEnv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
